# Generation of HD LiDAR Map using Depth Estimates from Depth Anything

In [ ]:
import glob
import os
from typing import Optional

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import open3d as o3d
import scipy
import trimesh
import vdbfusion
from tqdm import tqdm
from trimesh import transform_points
from vdbfusion import VDBVolume

## 1. Depth Estimation using Depth Anything

In [ ]:
!python Depth-Anything/run.py --encoder vitb --img-path data/kitti-odometry/sequences/00/image_2 --outdir data/kitti-odometry/sequences/00/depth --pred-only --grayscale

## 2. Creation of Local Point Clouds

In [ ]:
def read_projection_matrix(filepath):
    with open(filepath, 'r') as file:
        for line in file:
            if line.startswith("P2:"):
                p2_values = list(map(float, line.split()[1:]))
                P = np.array(p2_values).reshape(3, 4)
                return P

def decompose_projection_matrix(P):
    M = P[:, :3]
    p = P[:, 3]
    K, R = scipy.linalg.rq(M)
    K /= K[2, 2]
    t = np.linalg.inv(K) @ p
    return K, R, t

In [ ]:
P = read_projection_matrix('data/kitti-odometry/sequences/00/calib.txt')
K, R, t = decompose_projection_matrix(P)

In [ ]:
def get_projected_points(depth_image):
    img = cv2.imread(depth_image, 0)

    u, v = np.meshgrid(np.arange(img.shape[1]), np.arange(img.shape[0]))
    u = u.flatten()
    v = np.flipud(v.flatten())
    depth = img.flatten()

    u = u[depth > 0]
    v = v[depth > 0]
    depth = depth[depth > 0]

    return np.stack((u * depth, v * depth, depth), axis=-1).astype(np.float32)

In [ ]:
for idx in range(100):
    projected_points = get_projected_points(f'data/kitti-odometry/sequences/00/depth/{idx:06}_depth.png')
    points = projected_points @ np.linalg.inv(K).T
    points = (points - t) @ R
    points = np.hstack([points, np.ones((points.shape[0], 1))]).astype(np.float32)
    points.tofile(f'data/kitti-odometry/sequences/00/velodyne/{idx:06}.bin')

## 3. Pose Estimation using KISS-ICP

In [ ]:
!kiss_icp_pipeline --dataloader kitti --sequence 0 data/kitti-odometry/

## 4. Dataloader for Transformed Point Clouds

In [ ]:
# Borrowed (with modifications) from VDBFusion, https://github.com/PRBonn/vdbfusion/blob/main/examples/notebooks/kitti_odometry.ipynb, subject to the following terms.

# MIT License

# Copyright (c) 2022 Ignacio Vizzo, Cyrill Stachniss, University of Bonn

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

class Config:
    # Data specific params
    apply_pose = True
    min_range = 2.0
    max_range = 70.0


class KITTIOdometryDataset:
    def __init__(self, kitti_root_dir: str, sequence: int, pose_estimate: Optional[str] = None):
        """Simple KITTI DataLoader to provide a ready-to-run example.

        Heavily inspired in PyLidar SLAM
        """
        # Config stuff
        self.sequence = str(int(sequence)).zfill(2)
        self.config = Config()
        self.kitti_sequence_dir = os.path.join(kitti_root_dir, "sequences", self.sequence)
        self.velodyne_dir = os.path.join(self.kitti_sequence_dir, "velodyne/")

        # Read stuff
        self.calibration = self.read_calib_file(os.path.join(self.kitti_sequence_dir, "calib.txt"))
        if pose_estimate is None:
            self.poses = self.load_poses(os.path.join(kitti_root_dir, f"poses/{self.sequence}.txt"))
        else:
            self.poses = np.load(pose_estimate)
            self.poses = self._lidar_pose_gt(self.poses)
        self.scan_files = sorted(glob.glob(self.velodyne_dir + "*.bin"))

    def __getitem__(self, idx):
        return self.scans(idx), self.poses[idx]

    def __len__(self):
        return len(self.scan_files)

    def scans(self, idx):
        return self.read_point_cloud(idx, self.scan_files[idx], self.config)

    def read_point_cloud(self, idx: int, scan_file: str, config: Config):
        points = np.fromfile(scan_file, dtype=np.float32).reshape((-1, 4))[:, :-1]
        points = points[np.linalg.norm(points, axis=1) <= config.max_range]
        points = points[np.linalg.norm(points, axis=1) >= config.min_range]
        points = transform_points(points, self.poses[idx]) if config.apply_pose else points
        return points

    def load_poses(self, poses_file):
        poses = pd.read_csv(poses_file, sep=" ", header=None).values
        n = poses.shape[0]
        poses = np.concatenate(
            (poses, np.zeros((n, 3), dtype=np.float32), np.ones((n, 1), dtype=np.float32)), axis=1
        )
        poses = poses.reshape((n, 4, 4))  # [N, 4, 4]
        return self._lidar_pose_gt(poses)

    def _lidar_pose_gt(self, poses_gt):
        _tr = self.calibration["Tr"].reshape(3, 4)
        tr = np.eye(4, dtype=np.float64)
        tr[:3, :4] = _tr
        left = np.einsum("...ij,...jk->...ik", np.linalg.inv(tr), poses_gt)
        right = np.einsum("...ij,...jk->...ik", left, tr)
        return right

    @staticmethod
    def read_calib_file(file_path: str) -> dict:
        calib_dict = {}
        with open(file_path, "r") as calib_file:
            for line in calib_file.readlines():
                tokens = line.split(" ")
                if tokens[0] == "calib_time:":
                    continue
                # Only read with float data
                if len(tokens) > 0:
                    values = [float(token) for token in tokens[1:]]
                    values = np.array(values, dtype=np.float32)
                    # The format in KITTI's file is <key>: <f1> <f2> <f3> ...\n -> Remove the ':'
                    key = tokens[0][:-1]
                    calib_dict[key] = values
        return calib_dict

## 5. Generation of LiDAR Map

In [ ]:
dataset = KITTIOdometryDataset(kitti_root_dir='data/kitti-odometry/', sequence=0, \
                                                    pose_estimate='results/latest/00_poses.npy')

points = []
for scan, pose in tqdm(dataset):
    points.append(scan)

points = np.vstack(points)
points.tofile('data/kitti-odometry/map/map.bin')

## 6. Visualization of LiDAR Map, integrated through VDBFusion

In [ ]:
vdb_volume = VDBVolume(voxel_size=0.1, sdf_trunc=0.3, space_carving=False)
for scan, pose in tqdm(dataset):
    vdb_volume.integrate(scan, pose)

vertices, triangles = vdb_volume.extract_triangle_mesh(fill_holes=True, min_weight=5.0)
mesh = trimesh.Trimesh(vertices=vertices, faces=triangles)
mesh.show()